#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [352]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.





In [353]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [354]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [355]:
data = pd.read_csv('/content/drive/MyDrive/KT_aivle/2024.03.27_미니프로젝트 3차_실습자료/data01_train.csv' )

In [356]:
data.drop('subject',axis=1, inplace=True)

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [357]:
data['Activity_dynamic'] = np.where((data.Activity == 'STANDING') | (data.Activity == 'SITTING') | (data.Activity == 'LAYING'), 0, 1)
data['Activity_dynamic'].value_counts()

0    3234
1    2647
Name: Activity_dynamic, dtype: int64

In [358]:
x = data.drop(['Activity_dynamic','Activity'], axis=1)
y1 = data['Activity']
y2 = data['Activity_dynamic']

In [359]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x = scaler.fit_transform(x)

In [360]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y2,stratify=y2,test_size=.2)

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [361]:
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


#### 1) 알고리즘1 :

In [362]:
from sklearn.ensemble import RandomForestClassifier
model_dynamics=RandomForestClassifier(random_state=1)
model_dynamics.fit(x_train, y_train)
pred = model_dynamics.predict(x_val)

In [363]:
from sklearn.metrics import *

print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       647
           1       1.00      1.00      1.00       530

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



#### 2) 알고리즘2 :

In [364]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression(random_state=1)
model.fit(x_train, y_train)
pred = model.predict(x_val)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [365]:
from sklearn.metrics import *

print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       647
           1       1.00      1.00      1.00       530

    accuracy                           1.00      1177
   macro avg       1.00      1.00      1.00      1177
weighted avg       1.00      1.00      1.00      1177



### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [366]:
static = data.loc[data['Activity_dynamic']==0]

In [367]:
x = static.drop(['Activity_dynamic','Activity'], axis=1)
y1 = static['Activity']
y2 = static['Activity_dynamic']

In [368]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y1,stratify=y1,test_size=.2)

In [369]:
y1.unique()

array(['STANDING', 'LAYING', 'SITTING'], dtype=object)

In [370]:
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

In [371]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
pred = model.predict(x_val)

In [372]:
from sklearn.metrics import *

print(classification_report(y_val, pred))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       223
     SITTING       0.94      0.97      0.95       206
    STANDING       0.97      0.94      0.95       218

    accuracy                           0.97       647
   macro avg       0.97      0.97      0.97       647
weighted avg       0.97      0.97      0.97       647



In [373]:
from sklearn.svm import SVC
model_ststic=SVC(kernel='rbf',C=100.0,random_state=1)
model_ststic.fit(x_train, y_train)
pred = model_ststic.predict(x_val)

In [374]:
from sklearn.metrics import *

print(classification_report(y_val, pred))

              precision    recall  f1-score   support

      LAYING       1.00      1.00      1.00       223
     SITTING       0.97      0.98      0.97       206
    STANDING       0.98      0.97      0.97       218

    accuracy                           0.98       647
   macro avg       0.98      0.98      0.98       647
weighted avg       0.98      0.98      0.98       647



### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

In [400]:
static = data.loc[data['Activity_dynamic']==1]

In [401]:
x = static.drop(['Activity_dynamic','Activity'], axis=1)
y1 = static['Activity']
y2 = static['Activity_dynamic']

In [402]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x,y1,stratify=y1,test_size=.2)

In [403]:
x_train = scaler.transform(x_train)
x_val = scaler.transform(x_val)

In [398]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(random_state=1)
model.fit(x_train, y_train)
pred = model.predict(x_val)

In [399]:
from sklearn.metrics import *

print(classification_report(y_val, pred))

                    precision    recall  f1-score   support

           WALKING       0.98      0.99      0.98       200
WALKING_DOWNSTAIRS       0.98      0.97      0.97       158
  WALKING_UPSTAIRS       1.00      0.99      1.00       172

          accuracy                           0.98       530
         macro avg       0.99      0.98      0.98       530
      weighted avg       0.98      0.98      0.98       530



In [404]:
from sklearn.svm import SVC
model_dynamic=SVC(kernel='rbf',C=100.0,random_state=1)
model_dynamic.fit(x_train, y_train)
pred = model_dynamics.predict(x_val)

In [405]:
from sklearn.metrics import *

print(classification_report(y_val, pred))

                    precision    recall  f1-score   support

           WALKING       1.00      1.00      1.00       200
WALKING_DOWNSTAIRS       1.00      1.00      1.00       158
  WALKING_UPSTAIRS       1.00      1.00      1.00       172

          accuracy                           1.00       530
         macro avg       1.00      1.00      1.00       530
      weighted avg       1.00      1.00      1.00       530



### [선택사항] (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들어서 분류 모델 합치기

In [414]:
test = pd.read_csv('/content/drive/MyDrive/KT_aivle/2024.03.27_미니프로젝트 3차_실습자료/data01_test.csv')

In [415]:
test.drop('subject',axis=1,inplace=True)

In [422]:
test['Activity_dynamic'] = np.where((test.Activity == 'STANDING') | (test.Activity == 'SITTING') | (test.Activity == 'LAYING'), 0, 1)
test['Activity_dynamic'].value_counts()

0    833
1    638
Name: Activity_dynamic, dtype: int64

In [442]:
x_test = test.drop(['Activity','Activity_dynamic'], axis=1)
y_test_dynamic = test['Activity_dynamic']
y_test = test['Activity']

In [441]:
y_test

0                  SITTING
1                 STANDING
2                  WALKING
3                  SITTING
4                 STANDING
               ...        
1466               SITTING
1467              STANDING
1468    WALKING_DOWNSTAIRS
1469                LAYING
1470    WALKING_DOWNSTAIRS
Name: Activity, Length: 1471, dtype: object

In [443]:
x_test_s = scaler.transform(x_test)

In [444]:
pred_dynamics = model_dynamics.predict(x_test_s)
print(classification_report(y_test_dynamic, pred_dynamics))

ValueError: Mix of label input types (string and number)

In [438]:
x_test['Activity_dynamic'] = pred_dynamis

In [439]:
x_test_dymanic = x_test.loc[x_test['Activity_dynamic']>.5]
x_test_static = x_test.loc[x_test['Activity_dynamic']<.5]

TypeError: '>' not supported between instances of 'str' and 'float'

In [429]:
x_test_dymanic_s=x_test_dymanic.drop('Activity_dynamic',axis=1)
x_test_dymanic_s = scaler.transform(x_test_dymanic_s)

x_test_static_s=x_test_static.drop('Activity_dynamic',axis=1)
x_test_static_s = scaler.transform(x_test_static_s)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- pred


In [430]:
pred_dy = model_dynamic.predict(x_test_dymanic_s)
pred_st = model_ststic.predict(x_test_static_s)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but SVC was fitted without feature names
  warnings.warn(


ValueError: could not convert string to float: 'LAYING'

In [ ]:
x_test_dymanic['pred'] = pred_dy
x_test_static['pred'] = pred_st

In [ ]:
data_end = x_test_dymanic.append(x_test_static)

In [ ]:
data_end.sort_index(inplace=True)


In [ ]:
print(classification_report(y_test, data_end['pred']))


In [ ]:
y_test,data_end['pred']